In [1]:
# !pip install calitp-data-analysis==2025.6.24
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
import geopandas as gpd
import pandas as pd
import numpy as np

from shared_utils import webmap_utils, catalog_utils
from calitp_data_analysis import geography_utils
import branca

gcsgp = GCSGeoPandas()

# Webmap Examples

These maps are useful for displaying large datasets with better performance than Folium (as done in Speed Maps), or sharing draft datasets with just a single link.

## Basic Map

* use `webmap_utils.set_state_export`

In [2]:
shared_data = catalog_utils.get_catalog('shared_data_catalog')

In [3]:
stops = (shared_data.ca_transit_stops
         .read()
         .to_crs(geography_utils.CA_NAD83Albers_m)
         .query('agency == "Los Angeles County Metropolitan Transportation Authority"'))

stops.geometry = stops.buffer(40) #  points are currently hard to see at some zoom levels

In [4]:
webmap_utils.set_state_export?

Signature:
webmap_utils.set_state_export(
    gdf,
    bucket: str = 'calitp-map-tiles/',
    subfolder: str = 'testing/',
    filename: str = 'test2',
    map_type=None,
    map_title: str = 'Map',
    cmap: branca.colormap.ColorMap = None,
    color_col: str = None,
    legend_url: str = None,
    existing_state: dict = {},
    cache_seconds: int = 3600,
    manual_centroid: list = None,
) -> dict
Docstring:
Main function to use single page application webmap.

Applies light formatting to gdf for successful spa display. Will pass map_type
if supported by the spa and provided, otherwise leave as None for generic style.
GCS bucket is preset to a publically available one.
Supply cmap and color_col for coloring based on a Branca ColorMap and a column
to apply the color to.
Cache is 1 hour by default, can set shorter time in seconds for
"near realtime" applications (suggest 120) or development (suggest 0)

Returns dict with state dictionary and map URL. Can call multiple times and supply


### Tips

Always specify `filename` and `map_title`, also add subfolder outside the testing context. `map_type` gives custom styling control, but requires [implementation in the web app itself](https://github.com/cal-itp/data-infra/blob/main/apps/maps/src/routes/%2Bpage.svelte) and is best used for ongoing projects.

Default style mouseovers display all columns and can run off the screen, consider reducing column count or shortening values.

In [30]:
basic_state = webmap_utils.set_state_export(stops, filename='test_stops0', map_title='(Some) CA Transit Stops')

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:117: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [31]:
basic_state

{'state_dict': {'name': 'null',
  'layers': [{'name': '(Some) CA Transit Stops',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/test_stops0.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': (34.05874581954676, -118.3094728575273),
  'zoom': 13},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICIoU29tZSkgQ0EgVHJhbnNpdCBTdG9wcyIsICJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvdGVzdGluZy90ZXN0X3N0b3BzMC5nZW9qc29uLmd6IiwgInByb3BlcnRpZXMiOiB7InN0cm9rZWQiOiBmYWxzZSwgImhpZ2hsaWdodF9zYXR1cmF0aW9uX211bHRpcGxpZXIiOiAwLjV9fV0sICJsYXRfbG9uIjogWzM0LjA1ODc0NTgxOTU0Njc2LCAtMTE4LjMwOTQ3Mjg1NzUyNzNdLCAiem9vbSI6IDEzfQ=='}

`spa_link` is a live link to your map!

`webmap_utils` also includes some helpers to show links or embed your map in a notebook.

In [32]:
webmap_utils.render_spa_link(basic_state['spa_link'], text="My New Map")

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICIoU29tZSkgQ0EgVHJhbnNpdCBTdG9wcyIsICJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvdGVzdGluZy90ZXN0X3N0b3BzMC5nZW9qc29uLmd6IiwgInByb3BlcnRpZXMiOiB7InN0cm9rZWQiOiBmYWxzZSwgImhpZ2hsaWdodF9zYXR1cmF0aW9uX211bHRpcGxpZXIiOiAwLjV9fV0sICJsYXRfbG9uIjogWzM0LjA1ODc0NTgxOTU0Njc2LCAtMTE4LjMwOTQ3Mjg1NzUyNzNdLCAiem9vbSI6IDEzfQ==" target="_blank">Open My New Map in New Tab</a>

In [33]:
webmap_utils.display_spa_map(basic_state['spa_link'])

## Adding a colorscale

* That worked, but the gray default color does not stand out very well. Let's try something different.
* First, pick a branca colormap. We've mostly tested step colormaps so far. Scale it to your data or desired display

In [9]:
cmap = branca.colormap.step.Spectral_10.scale(vmin=0, vmax=5000)

In [10]:
cmap

In [11]:
color_state = webmap_utils.set_state_export(stops, filename='test_stops5', map_title='LA Metro Stops by Distance to SHS',
                                             cmap=cmap, color_col='meters_to_ca_state_highway')

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:117: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [12]:
color_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'LA Metro Stops by Distance to SHS',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/test_stops5.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': (34.05874581954676, -118.3094728575273),
  'zoom': 13},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJMQSBNZXRybyBTdG9wcyBieSBEaXN0YW5jZSB0byBTSFMiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdF9zdG9wczUuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX1dLCAibGF0X2xvbiI6IFszNC4wNTg3NDU4MTk1NDY3NiwgLTExOC4zMDk0NzI4NTc1MjczXSwgInpvb20iOiAxM30='}

In [13]:
webmap_utils.render_spa_link(color_state['spa_link'], text="My New Map")

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJMQSBNZXRybyBTdG9wcyBieSBEaXN0YW5jZSB0byBTSFMiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdF9zdG9wczUuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX1dLCAibGF0X2xvbiI6IFszNC4wNTg3NDU4MTk1NDY3NiwgLTExOC4zMDk0NzI4NTc1MjczXSwgInpvb20iOiAxM30=" target="_blank">Open My New Map in New Tab</a>

In [14]:
webmap_utils.display_spa_map(color_state['spa_link'])

## Adding a legend

* Legends are not automatically included in the webmap, but can be sourced as an image
* `webmap_utils.export_legend` helps convert a Branca colormap to a well-formatted svg
* Note that `inner_labels`, if specified, must be chosen manually. `cmap.index` is a helpful way to do this while referencing the number of steps in the colormap and its min/max
* Also note that the download link will only download the top layer

In [15]:
cmap.index

[0.0,
 500.0,
 1000.0,
 1500.0,
 2000.0,
 2500.0,
 3000.0,
 3500.0,
 4000.0,
 4500.0,
 5000.0]

In [16]:
cmap.caption = "Distance to State Highway System (meters)"

In [17]:
inner_labels = [1000, 2000, 3000, 4000]

In [18]:
webmap_utils.export_legend?

Signature:
webmap_utils.export_legend(
    cmap: branca.colormap.StepColormap,
    filename: str,
    inner_labels: list = [],
)
Docstring:
Given a branca colormap, export its html and reformat for successful display in webmap.

inner_labels is optional, but if provided should be four labels for correct spacing.
File:      ~/data-analyses/_shared_utils/shared_utils/webmap_utils.py
Type:      function

In [19]:
webmap_utils.export_legend(cmap=cmap, filename='demo_shs_distance.svg', inner_labels=inner_labels)

legend written to calitp-map-tiles/demo_shs_distance.svg, public_url https://storage.googleapis.com/calitp-map-tiles/demo_shs_distance.svg


In [20]:
legend_state = webmap_utils.set_state_export(stops, filename='test_stops5', map_title='LA Metro Stops by Distance to SHS',
                                             cmap=cmap, color_col='meters_to_ca_state_highway',
                                            legend_url="https://storage.googleapis.com/calitp-map-tiles/demo_shs_distance.svg")

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:117: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [21]:
legend_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'LA Metro Stops by Distance to SHS',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/test_stops5.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': (34.05874581954676, -118.3094728575273),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/demo_shs_distance.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJMQSBNZXRybyBTdG9wcyBieSBEaXN0YW5jZSB0byBTSFMiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdF9zdG9wczUuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX1dLCAibGF0X2xvbiI6IFszNC4wNTg3NDU4MTk1NDY3NiwgLTExOC4zMDk0NzI4NTc1MjczXSwgInpvb20iOiAxMywgImxlZ2VuZF91cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvZGVtb19zaHNfZGlzdGFuY2Uuc3ZnIn0='}

In [22]:
webmap_utils.render_spa_link(legend_state['spa_link'], text="My New Map")

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJMQSBNZXRybyBTdG9wcyBieSBEaXN0YW5jZSB0byBTSFMiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdF9zdG9wczUuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX1dLCAibGF0X2xvbiI6IFszNC4wNTg3NDU4MTk1NDY3NiwgLTExOC4zMDk0NzI4NTc1MjczXSwgInpvb20iOiAxMywgImxlZ2VuZF91cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvZGVtb19zaHNfZGlzdGFuY2Uuc3ZnIn0=" target="_blank">Open My New Map in New Tab</a>

In [23]:
webmap_utils.display_spa_map(legend_state['spa_link'])

## Adding more layers

* Adding layers is easy, simply pass a state dictionary containing previous layers to `set_state_export`
* For a solid color, create a `color` column in your gdf and assign a tuple with RGB or RGBA color values
* Note that `legend_url` must always be provided since it is global to the map and not assigned to a particular layer

In [24]:
cc_routes = shared_data.ca_transit_routes.read().query('agency == "City of Culver City"').to_crs(geography_utils.CA_NAD83Albers_m)
cc_routes.geometry = cc_routes.buffer(20)

In [25]:
cc_routes['color'] = [(15, 217, 35)] * cc_routes.shape[0]

In [26]:
cc_state = webmap_utils.set_state_export(cc_routes, filename='test_cc_routes1', map_title='LA Metro Stops by Distance to SHS, CC Routes in Green',
                                            legend_url="https://storage.googleapis.com/calitp-map-tiles/demo_shs_distance.svg",
                                        existing_state=legend_state)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:117: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [27]:
cc_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'LA Metro Stops by Distance to SHS',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/test_stops5.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}},
   {'name': 'LA Metro Stops by Distance to SHS, CC Routes in Green',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/test_cc_routes1.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': (34.009066013947994, -118.40483799612745),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/demo_shs_distance.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJMQSBNZXRybyBTdG9wcyBieSBEaXN0YW5jZSB0byBTSFMiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdF9zdG9wczUuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHR

In [28]:
webmap_utils.render_spa_link(cc_state['spa_link'], text="My New Map")

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJMQSBNZXRybyBTdG9wcyBieSBEaXN0YW5jZSB0byBTSFMiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdF9zdG9wczUuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX0sIHsibmFtZSI6ICJMQSBNZXRybyBTdG9wcyBieSBEaXN0YW5jZSB0byBTSFMsIENDIFJvdXRlcyBpbiBHcmVlbiIsICJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvdGVzdGluZy90ZXN0X2NjX3JvdXRlczEuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX1dLCAibGF0X2xvbiI6IFszNC4wMDkwNjYwMTM5NDc5OTQsIC0xMTguNDA0ODM3OTk2MTI3NDVdLCAiem9vbSI6IDEzLCAibGVnZW5kX3VybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy9kZW1vX3Noc19kaXN0YW5jZS5zdmcifQ==" target="_blank">Open My New Map in New Tab</a>

In [29]:
webmap_utils.display_spa_map(cc_state['spa_link'])